# ***overview***:
We decided to achieve the topic modeling task in a jypter notebook , because we need to manualley select the topics that are relevant to our task.

### ***import libraries***:


In [1]:
! pip install bertopic 
import os
import nltk 
import datetime
import bertopic
import pandas as pd
from bertopic import BERTopic
from google.colab import drive
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA 
nltk.download('vader_lexicon')
sia = SIA()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


### ***import Data***:


In [4]:
aljazerra_cleaned_data = pd.read_csv("../../data_extraction/cleaned_data/aljazerra_cleaned_data.csv")
aljazerra_uncleaned_data = pd.read_csv("../../data_extraction/raw_data/aljazerra_data.csv")
bbc_cleaned_data = pd.read_csv("../../data_extraction/cleaned_data/bbc_cleaned_data.csv")
bbc_uncleaned_data = pd.read_csv("../../data_extraction/raw_data/bbc_data.csv")
guardian_cleaned_data = pd.read_csv("../../data_extraction/cleaned_data/guardian_cleaned_data.csv")
#guardian_uncleaned_data = pd.read_csv("guardian.CSV")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/kaggledataset


### ***General sentiment statistics***:

In [10]:
def generate_general_statistical_sentiment_scores(data):
    sentiment_scores_general = {'positive_mean':data['title_main_content_positive_mean'].mean(), 'negatives_mean':data['title_main_content_negaitve_mean'].mean(), 'neutral_mean': data['title_main_content_neutral_mean'].mean()}
    return sentiment_scores_general
       

def generate_title_statistical_sentiment_scores(data):
    sentiment_scores_titles = {'positive_mean': data['title_positive_scores'].mean(), 'negatives_mean': data['title_negative_scores'].mean(), 'neutral_mean':data['title_neutral_scores'].mean()}
    return sentiment_scores_titles 

### ***Topic modeling***:


#### 1) Aljazerra 

In [89]:
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 7)
headline_topics, _ = model.fit_transform(aljazerra_cleaned_data.main_content) 

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

2023-03-13 14:58:58,178 - BERTopic - Transformed documents to Embeddings
2023-03-13 14:59:02,395 - BERTopic - Reduced dimensionality
2023-03-13 14:59:02,433 - BERTopic - Clustered reduced embeddings


In [114]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head(40)

Number of topics: 27


,Topic,Count,Name
0,-1,266,-1_al_tournament_also_one
1,0,67,0_morocco_moroccan_atlas_belgium
2,1,41,1_messi_argentina_lionel_final
3,2,37,2_croatia_japan_belgium_canada
4,3,35,3_november_group_best_appearances
5,4,34,4_doha_al_tea_country
6,5,31,5_cameroon_brazil_serbia_switzerland
7,6,30,6_group_ecuador_senegal_pm
8,7,28,7_host_tournament_first_teams
9,8,26,8_england_forwards_defenders_kane


In [91]:
aljazerra_cleaned_data['cluster'] = headline_topics

#### Sentiment analysis on the following topics:


*   Corruption allegations:
    (investigation_ibhais_committee_officials)
*   Rights of migrant workers: (workers_labour_migrant_rights)







**Sentiment functions**

 1) Investigations about allegations of bribery:
 (investigation_ibhais_committee_officials)

In [93]:
aljazerra_allegations_of_corruption = aljazerra_cleaned_data.loc[aljazerra_cleaned_data["cluster"]==20]
aljazerra_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(aljazerra_allegations_of_corruption)
aljazerra_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(aljazerra_allegations_of_corruption)
print("The general sentiment scores (the average of the title and main content) of Al Jazeera regarding the subtopic 'allegations of corruption' are "": \n {}".format(aljazerra_sentiment_scores_general_dictionary))
print("The titles sentiment scores of  Al Jazeera regarding the subtopic 'allegations of corruption' are: \n {} ".format(aljazerra_sentiment_scores_titles_dictionary))

The general sentiment scores (the average of the title and main content) of Al Jazeera regarding the subtopic 'allegations of corruption' are : 
 {'positive_mean': 0.08487499999999999, 'negatives_mean': 0.113, 'neutral_mean': 0.7828333333333334}
The titles sentiment scores of  Al Jazeera regarding the subtopic 'allegations of corruption' are: 
 {'positive_mean': 0.0655, 'negatives_mean': 0.17758333333333334, 'neutral_mean': 0.7569166666666667} 


In [42]:
merged_df = pd.merge(aljazerra_allegations_of_corruption, aljazerra_uncleaned_data, on='ID')
merged_df.to_csv("aljazeera_subtopic_corruption_allegations_data.csv")

 2) migrant workers rights:
 (workers_labour_migrant_rights)

In [95]:
aljazerra_workers_rights = aljazerra_cleaned_data.loc[(aljazerra_cleaned_data["cluster"]==13)]
aljazerra_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(aljazerra_workers_rights)
aljazerra_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(aljazerra_workers_rights)
print("The general sentiment scores (the average of the title and main content) of Al Jazeera reagrding the subtopic 'worker rights' are: \n {}".format(aljazerra_sentiment_scores_general_dictionary))
print("The titles sentiment scores of Al jazerra regarding the subtopic 'worker rights' are: \n {} ".format(aljazerra_sentiment_scores_titles_dictionary))

The general sentiment scores (the average of the title and main content) of Al Jazeera reagrding the subtopic 'worker rights' are: 
 {'positive_mean': 0.14409375, 'negatives_mean': 0.1033125, 'neutral_mean': 0.742}
The titles sentiment scores of Al jazerra regarding the subtopic 'worker rights' are: 
 {'positive_mean': 0.1333125, 'negatives_mean': 0.1714375, 'neutral_mean': 0.6952499999999999} 


In [69]:
merged_df = pd.merge(aljazerra_workers_rights, aljazerra_uncleaned_data, on='ID')
merged_df.to_csv("aljazerra_subtopic_worker_rights_data.csv")

#### 1) BBC 

In [ ]:
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 3)
headline_topics, _ = model.fit_transform(bbc_cleaned_data.main_content)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2023-03-06 17:30:53,038 - BERTopic - Transformed documents to Embeddings
2023-03-06 17:30:59,994 - BERTopic - Reduced dimensionality
2023-03-06 17:31:00,017 - BERTopic - Clustered reduced embeddings


In [ ]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head(27)

Number of topics: 20


,Topic,Count,Name
0,-1,45,-1_people_one_rights_would
1,0,43,0_lgbt_rights_gay_human
2,1,15,1_saudi_arabia_al_gulf
3,2,13,2_game_iran_england_win
4,3,11,3_airport_cardiff_tenerife_get
5,4,11,4_algeria_moroccan_final_minutes
6,5,9,5_neville_itv_russia_bbc
7,6,9,6_tournament_players_infantino_years
8,7,9,7_workers_migrant_labour_ibhais
9,8,8,8_welsh_song_hyd_yma


In [ ]:
bbc_cleaned_data['cluster'] = headline_topics
bbc_cleaned_data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,title,main_content,ID,cluster
0,0,0,0,14 February 2023\n14 February 2023,turkey syria earthquakes send mobile homes aff...,sending mobile homes used accommodation areas ...,0,-1
1,1,2,2,23 January,gary neville ofcom pursuing complaints comments,ofcom pursuing complaints received itv viewers...,2,5
2,2,3,3,5 January,drakeford regrets hotel freebie,mark drakeford says emphasised human rights me...,3,0
3,3,4,4,5 January,mark drakeford defends taking hospitality way,first minister mark drakeford says choice stay...,4,0
4,4,6,6,19 December 2022,rishi sunak wades gary neville workers rights row,audiences want hear gary neville discuss polit...,6,5


#### Sentiment analysis on the following topics:


*   gay_rights:
    (lgbt_rights_gay_human)
*   Rights of migrant workers: (workers_labour_amnesty_migrant_rights_ibhais)
*   banning of alochol: (alcohol_minute_sale_ban)







1) gay_rights:
    (lgbt_rights_gay_human)

In [ ]:
bbc_gay_rights = bbc_cleaned_data.loc[bbc_cleaned_data["cluster"]==0]
bbc_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(bbc_gay_rights)
bbc_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(bbc_gay_rights)
print("The general sentiment scores of bbc reagrdig the gay rights are: \n {}".format(bbc_sentiment_scores_general_dictionary))
print("The titles sentiment scores of bbc regarding the gay rights are: \n {} ".format(bbc_sentiment_scores_titles_dictionary))

The general sentiment scores of bbc reagrdig the gay rights are: 
 {'positive_mean': 0.15081395348837212, 'negatives_mean': 0.12093023255813952, 'neutral_mean': 0.6914186046511628}
The titles sentiment scores of bbc regarding the gay rights are: 
 {'positive_mean': 0.11404651162790698, 'negatives_mean': 0.15527906976744188, 'neutral_mean': 0.7306976744186046} 


In [ ]:
merged_df = pd.merge(bbc_gay_rights_sentiment_df, bbc_uncleaned_data, on='ID')
merged_df.to_csv("bbc_subtopic_lgbtq_rights_data.csv")

2) Rights of migrant workers: (workers_labour_amnesty_migrant_rights_ibhais)

In [ ]:
bbc_worker_rights = bbc_cleaned_data.loc[(bbc_cleaned_data["cluster"]==7)| (bbc_cleaned_data["cluster"]== 9)]
bbc_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(bbc_worker_rights)
bbc_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(bbc_worker_rights)
print("The general sentiment scores of bbc reagrdig the worker rights are: \n {}".format(bbc_sentiment_scores_general_dictionary))
print("The titles sentiment scores of bbc regarding the gay rights are: \n {} ".format(bbc_sentiment_scores_titles_dictionary))

The general sentiment scores of bbc reagrdig the worker rights are: 
 {'positive_mean': 0.12743749999999998, 'negatives_mean': 0.1255, 'neutral_mean': 0.7080625}
The titles sentiment scores of bbc regarding the gay rights are: 
 {'positive_mean': 0.0884375, 'negatives_mean': 0.1411875, 'neutral_mean': 0.770375} 


In [ ]:
merged_df = pd.merge(bbc_worker_rights, bbc_uncleaned_data, on='ID')
merged_df.to_csv("bbc_subtopic_worker_rights_data.csv")

3) Banning of alcohol:(alcohol_minute_sale_ban)

In [ ]:
bbc_alcohol_ban = bbc_cleaned_data.loc[bbc_cleaned_data.cluster==17]
bbc_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(bbc_alcohol_ban)
bbc_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(bbc_alcohol_ban)
print("The general sentiment scores of bbc regarding the ban of alcohol are: \n {}".format(bbc_sentiment_scores_general_dictionary))
print("The titles sentiment scores of bbc regarding the ban of alcohol are: \n {} ".format(bbc_sentiment_scores_titles_dictionary))

The general sentiment scores of bbc reagrdig the ban of alcohol are: 
 {'positive_mean': 0.23775000000000002, 'negatives_mean': 0.13225, 'neutral_mean': 0.67675}
The titles sentiment scores of bbc regarding the ban of alcohol are: 
 {'positive_mean': 0.28475, 'negatives_mean': 0.38575000000000004, 'neutral_mean': 0.3295} 


In [ ]:
merged_df = pd.merge(bbc_alcohol_ban, bbc_uncleaned_data, on='ID')
merged_df.to_csv("bbc_subtopic_alcohol_data.csv")

#### 3) Guardian

In [ ]:
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 7)
headline_topics, _ = model.fit_transform(guardian_cleaned_data.main_content)

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

2023-03-06 14:12:33,568 - BERTopic - Transformed documents to Embeddings
2023-03-06 14:12:53,396 - BERTopic - Reduced dimensionality
2023-03-06 14:12:53,475 - BERTopic - Clustered reduced embeddings


In [ ]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head(45)

Number of topics: 37


,Topic,Count,Name
0,-1,496,-1_one_min_time_game
1,0,202,0_england_southgate_kane_back
2,1,176,1_workers_migrant_labour_rights
3,2,127,2_blatter_winter_platini_president
4,3,102,3_committee_garcia_bid_executive
5,4,82,4_messi_argentina_final_like
6,5,71,5_al_qatari_doha_workers
7,6,67,6_germany_japan_spain_game
8,7,55,7_morocco_moroccan_regragui_ghana
9,8,44,8_lgbtq_gay_rights_rainbow


#### Sentiment analysis on the following topics:


*   gay_rights:
    (lgbtq_gay_rights_rainbow)
*   Rights of migrant workers: (workers_migrant_labour_rights_qatari_doha_human_kit)
*   banning of alochol: (alcohol_budweiser_beer_fan)
*   corruption allegations: (committee_garcia_bid_executive_swiss_investigation_garcia_evidence)
* armband: (armband_wear_armbands_sanctions)

1) Rights of migrant workers: (workers_migrant_labour_rights_qatari_doha_human_kit)

In [ ]:
guardian_cleaned_data['cluster'] = headline_topics

In [ ]:
guardian_worker_rights = guardian_cleaned_data.loc[(guardian_cleaned_data.cluster== 1) | (guardian_cleaned_data.cluster== 5 ) | (guardian_cleaned_data.cluster== 27)]
guardian_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(guardian_worker_rights)
guardian_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(guardian_worker_rights)
print("The general sentiment scores of the guardian reagrdig the worker rights are: \n {}".format(guardian_sentiment_scores_general_dictionary))
print("The titles sentiment scores of the guardian regarding the worker rights are: \n {} ".format(guardian_sentiment_scores_titles_dictionary))
guardian_worker_rights_sentiment_df.to_csv("guardian_worker_rights.csv")

The general sentiment scores of the guardian reagrdig the worker rights are: 
 {'positive_mean': 0.12814671814671813, 'negatives_mean': 0.12846332046332049, 'neutral_mean': 0.7207258687258687}
The titles sentiment scores of the guardian regarding the worker rights are: 
 {'positive_mean': 0.10547104247104247, 'negatives_mean': 0.21384942084942085, 'neutral_mean': 0.6652355212355212} 


In [ ]:
merged_df = pd.merge(guardian_worker_rights, guardian_uncleaned_data, on='url')
merged_df.to_csv("guardian_subtopic_worker_rights_data.csv")

2) Rights of gay people:(lgbtq_gay_rights_rainbow)

In [ ]:
guardian_gay_rights = guardian_cleaned_data.loc[guardian_cleaned_data["cluster"]==8]
guardian_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(guardian_gay_rights)
guardian_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(guardian_gay_rights)
guardian_gay_rights_sentiment_df.to_csv("guardian_gay_rights.csv")
print("The general sentiment scores of the guardian reagrdig the gay rights are: \n {}".format(guardian_sentiment_scores_general_dictionary))
print("The titles sentiment scores of the guardian regarding the gay rights are: \n {} ".format(guardian_sentiment_scores_titles_dictionary))

The general sentiment scores of the guardian reagrdig the gay rights are: 
 {'positive_mean': 0.1671818181818182, 'negatives_mean': 0.11465909090909089, 'neutral_mean': 0.6890681818181819}
The titles sentiment scores of the guardian regarding the gay rights are: 
 {'positive_mean': 0.13811363636363636, 'negatives_mean': 0.1657045454545454, 'neutral_mean': 0.6961818181818181} 


In [ ]:
merged_df = pd.merge(guardian_gay_rights, guardian_uncleaned_data, on='url')
merged_df.to_csv("guardian_subtopic_gay_rights_data.csv")

Banning of alcohol:(alcohol_budweiser_beer_fan)

In [ ]:
guardian_alcohol_ban = guardian_cleaned_data.loc[guardian_cleaned_data["cluster"]==28]
guardian_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(guardian_alcohol_ban)
guardian_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(guardian_alcohol_ban)
#guardian_gay_rights.to_csv("guardian_gay_rights.csv")
print("The general sentiment scores of the guardian reagrdig the ban of alcohol are: \n {}".format(guardian_sentiment_scores_general_dictionary))
print("The titles sentiment scores of the guardian regarding ban of alcohol are: \n {} ".format(guardian_sentiment_scores_titles_dictionary))

The general sentiment scores of the guardian reagrdig the ban of alcohol are: 
 {'positive_mean': 0.17827272727272728, 'negatives_mean': 0.08609090909090909, 'neutral_mean': 0.7158181818181818}
The titles sentiment scores of the guardian regarding ban of alcohol are: 
 {'positive_mean': 0.1588181818181818, 'negatives_mean': 0.217, 'neutral_mean': 0.6241818181818182} 


In [ ]:
merged_df = pd.merge(guardian_alcohol_ban_sentiment_df, guardian_uncleaned_data, on='url')
merged_df.to_csv("guardian_alcohol_ban_new.csv")

committee_garcia_bid_executive:

In [ ]:
guardian_bid_allegations = guardian_cleaned_data.loc[(guardian_cleaned_data["cluster"]==3) |  (guardian_cleaned_data["cluster"]==30)]
guardian_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(guardian_bid_allegations)
guardian_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(guardian_bid_allegations)
#guardian_gay_rights.to_csv("guardian_gay_rights.csv")
print("The general sentiment scores of the guardian reagrdig the bid allegations are: \n {}".format(guardian_sentiment_scores_general_dictionary))
print("The titles sentiment scores of the guardian regarding bid allegations are: \n {} ".format(guardian_sentiment_scores_titles_dictionary))

The general sentiment scores of the guardian reagrdig the bid allegations are: 
 {'positive_mean': 0.10184374999999998, 'negatives_mean': 0.09875892857142857, 'neutral_mean': 0.7816964285714286}
The titles sentiment scores of the guardian regarding bid allegations are: 
 {'positive_mean': 0.08416964285714286, 'negatives_mean': 0.15141964285714285, 'neutral_mean': 0.7644196428571428} 


In [ ]:
merged_df = pd.merge(guardian_bid_allegations, guardian_uncleaned_data, on='url')
merged_df.to_csv("guardian_bid_allegations_very_new.csv")

armband_wear_armbands_sanctions

In [ ]:
guardian_arm_band = guardian_cleaned_data.loc[guardian_cleaned_data["cluster"]==17]
guardian_sentiment_df = generate_sentiment_scores(guardian_arm_band)
guardian_sentiment_scores_general_dictionary = generate_general_statistical_sentiment_scores(guardian_sentiment_df)
guardian_sentiment_scores_titles_dictionary = generate_title_statistical_sentiment_scores(guardian_sentiment_df)
#guardian_gay_rights.to_csv("guardian_gay_rights.csv")
print("The general sentiment scores of the guardian reagrdig the wear of the arm band are: \n {}".format(guardian_sentiment_scores_general_dictionary))
print("The titles sentiment scores of the guardian regarding the wear of the arm band are: \n {} ".format(guardian_sentiment_scores_titles_dictionary))

The general sentiment scores of the guardian reagrdig the wear of the arm band are: 
 {'positive_mean': 0.13549999999999998, 'negatives_mean': 0.11340909090909092, 'neutral_mean': 0.7151363636363637}
The titles sentiment scores of the guardian regarding the wear of the arm band are: 
 {'positive_mean': 0.0995, 'negatives_mean': 0.26886363636363636, 'neutral_mean': 0.6316363636363637} 


In [ ]:
merged_df = pd.merge(guardian_arm_band, guardian_uncleaned_data, on='url')
merged_df.to_csv("guardian_arm_band_new.csv")